In [29]:
import torch

In [40]:
x = torch.tensor([1, 2, 3, 4, 5])
y = torch.tensor([5, 6, 7, 8, 9])
z = torch.tensor([3, 4, 5, 6, 7])
p = torch.tensor([1, 7, 9, 6, 7])
q = torch.tensor([2, 6, 8, 6, 7])

In [45]:
torch.stack([x, y, z, p, q]).T.float().cuda().median(dim=1)

torch.return_types.median(
values=tensor([2., 6., 7., 6., 7.], device='cuda:0'),
indices=tensor([4, 1, 1, 2, 2], device='cuda:0'))

In [47]:
import torch


sample_shape_len = 4
all_axes = list(range(sample_shape_len))

for axis in [0, 1, 2, 3]:
    x = torch.randn(1,2,3,4)
    channels_permutation = (axis, *[x for x in all_axes if x != axis])
    permuted_x = torch.permute(x, channels_permutation)

    inverse_channels_permutation = list(range(1, axis + 1)) + [0] + all_axes[axis + 1 :]

    permuted_x_permuted = torch.permute(permuted_x, inverse_channels_permutation)

    assert x.shape == (1,2,3,4)
    print(x.shape, "   ->   ", permuted_x.shape, "  ->  ", permuted_x_permuted.shape)



torch.Size([1, 2, 3, 4])    ->    torch.Size([1, 2, 3, 4])   ->   torch.Size([1, 2, 3, 4])
torch.Size([1, 2, 3, 4])    ->    torch.Size([2, 1, 3, 4])   ->   torch.Size([1, 2, 3, 4])
torch.Size([1, 2, 3, 4])    ->    torch.Size([3, 1, 2, 4])   ->   torch.Size([1, 2, 3, 4])
torch.Size([1, 2, 3, 4])    ->    torch.Size([4, 1, 2, 3])   ->   torch.Size([1, 2, 3, 4])


In [1]:
!pwd

/home/shate/Desktop/praca_magisterska/medical_signal_representation/notebooks


In [2]:
import glob
import torch

In [6]:
files = sorted(glob.glob("../data/sleep_edf/representations/whole_signal_waveforms/train*"))

In [8]:
rep_split_tensor = torch.concatenate([torch.load(file) for file in files]).shape